In [4]:
# pip install --upgrade git+https://github.com/huggingface/transformers

In [4]:
#  pip install Image

In [10]:
from PIL import Image
import requests
from transformers import AutoProcessor, Kosmos2ForConditionalGeneration

model = Kosmos2ForConditionalGeneration.from_pretrained("microsoft/kosmos-2-patch14-224")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")

url = "https://huggingface.co/microsoft/kosmos-2-patch14-224/resolve/main/snowman.jpg"
image = Image.open(requests.get(url, stream=True).raw)

prompt = "<grounding> An image of"

inputs = processor(text=prompt, images=image, return_tensors="pt")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:

generated_ids = model.generate(
    pixel_values=inputs["pixel_values"],
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    image_embeds=None,
    image_embeds_position_mask=inputs["image_embeds_position_mask"],
    use_cache=True,
    max_new_tokens=64,
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
processed_text = processor.post_process_generation(generated_text, cleanup_and_extract=False)
processed_text

caption, entities = processor.post_process_generation(generated_text)
caption

print(entities)
print(caption)

# show the image and mark the entities
from IPython.display import display
from PIL import ImageDraw

draw = ImageDraw.Draw(image)
# for entity in entities:
#     draw.rectangle(entity["position"], outline="red", width=2)

# display(image)


[('a snowman', (12, 21), [(0.390625, 0.046875, 0.984375, 0.828125)]), ('a fire', (41, 47), [(0.171875, 0.015625, 0.484375, 0.890625)])]
An image of a snowman warming himself by a fire.


## Translator

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Replace 't5-base' with the specific model identifier you want to use
model_name = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Input text
input_text = "Translate this English sentence to French: Hello, how are you?"

# Tokenize input text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate output
output_ids = model.generate(input_ids)

# Decode output
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Input text:", input_text)
print("Translated text:", output_text)


Input text: Translate this English sentence to French: Hello, how are you?
Translated text: Bonjour, comment êtes-vous?
